In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import fasttext
from collections import Counter
import string
from utils.preprocessing import preprocess_func

In [ ]:
lemmatize = True
preprocess = preprocess_func(lemmatize=lemmatize)

In [ ]:
df = pd.concat([
    pd.read_csv('data/processed/ensisijainen.csv', index_col=0),
    pd.read_csv('data/processed/toissijainen.csv', index_col=0)    
])

In [ ]:
df.head()

In [ ]:
s = df['answer'].apply(preprocess)

In [ ]:
for a in s.values[:10]:
    print(a)

In [ ]:
c = Counter([t for answer in s for t in answer.split()])

In [ ]:
c.most_common(15)

In [ ]:
text = '\n'.join(s.values)

In [ ]:
esco_df = pd.read_csv('data/external/skills_fi.csv')
esco_df.head()

In [ ]:
esco_text = '\n'.join(esco_df['description'].apply(preprocess))

In [ ]:
with open('data/interim/fasttext_train_data', 'w') as f:
    f.write(text + esco_text)

In [ ]:
hparams = {
    'lr': 0.05,
    'dim': 30,
    'ws': 5,
    'epoch': 5,
    'minn': 4,
    'maxn': 6,
    'neg': 10,
    'loss': 'ns',
    'bucket': 2000000,    
    'lrUpdateRate': 100,
    't': 0.0001
}

rest = {
    'input': 'data/interim/fasttext_train_data',
    'model': 'skipgram',
    'minCount': 1,
    'minCountLabel': 0,
    'wordNgrams': 1,
    'thread': 1,
#     'label':
    'verbose': 5,
#     'pretrainedVectors':
}

model_name = 'fasttext-' + ','.join([f'{k}={v}' for k, v in hparams.items()] + [f'lemmatize={lemmatize}']) + '.bin'

In [ ]:
ft = fasttext.train_unsupervised(**hparams, **rest)

In [ ]:
model_name

In [ ]:
ft.save_model(os.path.join('models', model_name))

In [ ]:
ft.get_nearest_neighbors('lääkehoito', k=50)